In [1]:
class Location():
    
    def __init__(self, x=1, y=1):
        self.x = x
        self.y = y
        self.location = (self.x,self.y) 


import networkx as nx
from matplotlib import pyplot as plt

G = nx.grid_2d_graph(range(1,5), range(1,5))
print(G.nodes())
#plt.figure(figsize=(6,6))
pos = {(x,y):(x,y) for x,y in G.nodes()}
nx.draw(G, pos=pos, 
        node_color='lightgreen', 
        with_labels=True,
        node_size=600)
plt.show;

In [2]:
class WumpusEnvironment():
    
    def __init__(self, width = 4, height = 4, allow_climb_without_gold= True, pit_prob = 0):
        #we'll set pit probability to 0 for testing purposes to force agent to find gold
       
        self.grid_size = (width,height)
        self.grid = width
        self.x_start, self.y_start = (1, 1)
        self.x_end, self.y_end = (self.grid,self.grid)
        self.pit_prob = pit_prob
        self.allow_climb_without_gold = allow_climb_without_gold
        self.agent_loc = Location(self.x_start, self.y_start)
        self.agent_loc.x,self.agent_loc.y=self.agent_loc.x, self.agent_loc.y
        self.orientation = 'RIGHT'
        self.agent_alive = True
        self.agent_has_arrow = True
        self.agent_has_gold = False
        self.wumpus_alive = True
        self.scream = False
        self.make_things()
        self.make_percepts()
        self.wumpus_location = self.thing_position('wumpus')
        self.game_over = False
        self.score=0
        self.bump = False

        
    def make_things(self) -> tuple[dict,list,tuple]:
        from itertools import product
        import random
        self.cells =list(product((list(range(1,5))),repeat=2))
        self.things =  {key: [] for key in self.cells}
        self.pits = []
        
#create pits:
        for x in range(self.x_start, self.x_end + 1):
            for y in range(self.y_start, self.y_end+1):
                if (x != 1) or (y != 1):
                    if random.random() < self.pit_prob:
                        self.pits.append((x, y))
                        self.things[(x,y)].append('pit')
                        
#create wumpus
        self.wumpus=(random.randint(self.x_start, self.x_end), random.randint(self.y_start, self.y_end))
        while self.wumpus==(self.x_start,self.y_start):
            self.wumpus=(random.randint(self.x_start, self.x_end), random.randint(self.y_start, self.y_end))
        self.things[self.wumpus].append('wumpus')
                     
#create gold:
        self.gold=(random.randint(self.x_start, self.x_end), random.randint(self.y_start, self.y_end))
        while self.gold==(self.x_start,self.y_start):
            self.gold=(random.randint(self.x_start, self.x_end), random.randint(self.y_start, self.y_end))
        self.things[self.gold].append('gold')
             
        return self.things,self.pits, self.wumpus,self.gold
    

#to find location of wumpus and gold:
    def thing_position(self,thing) -> tuple:
        keys=list(self.things.keys())
        values=list(self.things.values())

        try:
            values.index([thing])
            position = values.index([thing])
        except:
            try: 
                for sublist in values:
                    if thing in sublist:
                        position = values.index(sublist)
            except:
                print('exception ocurred')
        return (keys[position])
    

    
#create percepts
    def same_loc (self,loc1, loc2) -> bool:
        return loc1[0] == loc2[0] and loc1[1] == loc2[1]
    
    
    def is_adjacent(self,loc1, loc2) -> bool:
        x1 = loc1[0]
        x2 = loc2[0]
        y1 = loc1[1]
        y2 = loc2[1]

        if (x1 == x2) and (y1 == (y2 - 1)) or \
           (x1 == x2) and (y1 == (y2 + 1)) or \
           (x1 == (x2 - 1)) and (y1 == y2) or \
           (x1 == (x2 + 1)) and (y1 == y2):
            return True
        return False
    
    
    def make_percepts(self) -> dict: 
        self.percepts = {key: [] for key in self.cells}
        for cell in self.percepts.keys():
            for pit in self.pits:
                if  self.is_adjacent(cell, pit) and 'breeze' not in self.percepts[cell]:
                    self.percepts[cell].append('breeze')


            if self.is_adjacent(cell,self.wumpus)or self.same_loc(cell,self.wumpus):
                self.percepts[cell].append('stench')

            if self.same_loc(cell,self.gold):
                self.percepts[cell].append('glitter')
        return self.percepts
        
    
#pass percepts to an agent for the cell it's in:    
    def get_percepts_and_score(self) -> list:
        import copy
        temp_percepts = copy.deepcopy(self.percepts[(self.agent_loc.x, self.agent_loc.y)])
        if self.bump:
            temp_percepts.append('bump')
            self.bump = False
        
        elif self.scream:
            temp_percepts.append('scream')
            self.scream = False
        temp_percepts.append(self.score)
        
       
        return temp_percepts


#to track how agent's actions affect the environment:
    def take_action(self,action):

            if action == 'forward':
                if self.orientation == 'RIGHT':
                    if  self.agent_loc.x < self.grid:
                        self.agent_loc.x += 1
                    else:
                        self.bump = True
                elif self.orientation == 'UP':
                    if self.agent_loc.y < self.grid:
                        self.agent_loc.y += 1
                    else:
                        self.bump = True

                elif self.orientation == 'LEFT':
                    if self.agent_loc.x > 1:
                        self.agent_loc.x -= 1
                    else:
                        self.bump = True

                elif self.orientation == 'DOWN':
                    if self.agent_loc.y > 1:
                        self.agent_loc.y -= 1
                    else:
                         self.bump = True      


            if action == 'turn_left':
                if self.orientation == 'RIGHT':
                    self.orientation = 'UP'
                elif self.orientation == 'UP':
                    self.orientation = 'LEFT'
                elif self.orientation == 'LEFT':
                    self.orientation = 'DOWN'
                elif self.orientation == 'DOWN':
                    self.orientation = 'RIGHT'

            if action == 'turn_right':
                if self.orientation == 'RIGHT':
                    self.orientation = 'DOWN'
                elif self.orientation == 'UP':
                    self.orientation = 'RIGHT'
                elif self.orientation == 'LEFT':
                    self.orientation = 'UP'
                elif self.orientation == 'DOWN':
                    self.orientation = 'LEFT'

            
            if action == 'grab':
                if not self.agent_has_gold and (self.agent_loc.x, self.agent_loc.y)==self.thing_position('gold'):
                    self.agent_has_gold = True   
                else:
                    print("Illegal action 'GRAB'")

            if action == 'shoot':
                if  self.agent_has_arrow:
                    self.agent_has_arrow = False
                    self.score-=10

                    if self.wumpus_alive:
                        if (((self.orientation == 'RIGHT') and
                             (self.agent_loc.x < self.wumpus_location[0]) and
                             (self.agent_loc.y == self.wumpus_location[1])) or

                            ((self.orientation == 'UP') and
                             (self.agent_loc.x == self.wumpus_location[0]) and
                             (self.agent_loc.y < self.wumpus_location[1])) or

                            ((self.orientation == 'LEFT') and
                             (self.agent_loc.x > self.wumpus_location[0]) and
                             (self.agent_loc.y == self.wumpus_location[1])) or

                            ((self.orientation == 'DOWN') and
                             (self.agent_loc.x == self.wumpus_location[0]) and
                             (self.agent_loc.y > self.wumpus_location[1]))):
                            self.wumpus_alive = False
                            self.scream = True

                else:
                    print('No arrows!')



            if action == 'climb':
                if self.agent_loc.x == self.x_start and\
                self.agent_loc.y == self.y_start and\
                self.agent_has_gold:
                    self.score+=1000
                    self.end_of_round()
                
                else:
                    print('Illegal action! You cannot climb without gold or outside of start square!')
                    
            if  ((self.agent_loc.x, self.agent_loc.y)==self.wumpus_location and self.wumpus_alive) or \
                ((self.agent_loc.x, self.agent_loc.y) in self.pits):
                    self.agent_alive = False
                    self.score-=1000
                    print(f"You've encountered a {'pit' if (self.agent_loc.x, self.agent_loc.y) in self.pits else 'wumpus'}!")
                    self.end_of_round()
            self.score-=1
                
   
    def end_of_round(self) ->int:
        
        if self.agent_alive and self.agent_has_gold:
            print("Congratulations! You won!")        
        else:
            print('Wa-Wa-Wa-Waaa! You lose!')
        self.game_over = True
        print(f"Final score: {self.score}")
        return self.score
         

#visualize the game:
    def print_world(self):
        """ print_world: print the current wumpus world"""

        # print out the first horizontal line
        out = "+"
        for x in range(1, self.grid + 1):
            out += "---+"
        print(out)

        for y in range(self.grid, 0, -1):  
            out = "|"

            for x in range(1, self.grid + 1):
                if self.wumpus_location[0] == x and self.wumpus_location[1]==y:
                    if self.wumpus_alive:
                        out += "W"
                    else:
                        out += "M"   #dead wumpus is upside down
                else:
                    out += " "

                if not self.agent_has_gold and self.thing_position('gold')[0] == x and\
                self.thing_position('gold')[1] ==y:
                    out += "G"
                else:
                    out += " "

                _has_pit = False
                for pit in self.pits:
                    if pit[0] == x and pit[1]==y:
                        _has_pit = True
                if _has_pit:
                    out += "P"
                else:
                    out += " "

                out += "|"

            print(out)
            out = "|"
            
#print the agent:
            for x in range(1, self.grid + 1):
                if self.agent_alive and self.agent_loc.x == x and self.agent_loc.y == y:
                    if self.orientation == 'RIGHT':
                        out += " A>|"
                    elif self.orientation == 'UP':
                        out += " A^|"
                    elif self.orientation == 'LEFT':
                        out += " A<|"
                    else:
                        out += " Av|"
                else:
                    out += "   |"

            print(out)
            out = "+"

            # print out the final horizontal line
            for x in range(1, self.grid + 1):
                out += "---+"

            print(out)

#print the current percepts and score for the agent's location
        print(self.get_percepts_and_score())
        print()

In [3]:
class BeelineAgent():
    def __init__(self,grid):
        import networkx as nx
        self.agent_type = 'beeline_agent'
        self.actions = ['forward', 'turn_left', 'turn_right']
        #,'climb',shoot']  We will remove some actions from the action list for testing purposes.
        #our agent is random and we want to force it to find gold occasionally
        self.latest_percept = []
        self.has_gold = False
        self.score = None
        self.agent_loc = Location(1,1)
        self.orientation = 'RIGHT'
        self.safe_locations = [(1,1)]
        self.latest_action = None
        self.grid = grid
        self.extra_turn = False
        self.target_loc = None
        self.graph = nx.grid_2d_graph(range(1,self.grid+1), range(1,self.grid+1)) #initialize 4x4 graph
    

#logic for choosing next action:    
    def next_action(self) -> str : 
        import random 
        if self.has_gold and self.agent_loc.x==1 and self.agent_loc.y==1:
            self.latest_action = 'climb'
        
        elif self.has_gold:
            
            if self.extra_turn or self.latest_action in ['turn_right', 'turn_left']:
                self.target_loc = self.target_loc
            else:
                self.target_loc = self.shortest_path.pop(0)
        
            self.target_orient=self.find_target_orientation\
            ((self.target_loc),(self.agent_loc.x,self.agent_loc.y) )  
            
            if self.target_orient==self.orientation:
                    self.extra_turn = False
                    self.latest_action = "forward"
                   
            elif self.target_orient!=self.orientation:
                self.latest_action = self.change_orientation(self.orientation, self.target_orient)
                if self.target_orient!=self.orientation:
                    self.extra_turn = True
                     
        elif not self.has_gold and 'glitter' in self.latest_percept:
            self.has_gold = True
            self.latest_action = 'grab'
            self.build_path_back()
                  
        else:
            self.latest_action = random.choice(self.actions)
        self.track_location()
        print(f"Selected action: {self.latest_action}")
        return self.latest_action
      

#to self-track agent's location:
    def track_location(self):
        if self.latest_action == 'forward':
                if self.orientation == 'RIGHT':
                    if  self.agent_loc.x < self.grid:
                        self.agent_loc.x += 1
                
                elif self.orientation == 'UP':
                    if self.agent_loc.y < self.grid:
                        self.agent_loc.y += 1

                elif self.orientation == 'LEFT':
                    if self.agent_loc.x > 1:
                        self.agent_loc.x -= 1

                elif self.orientation == 'DOWN':
                    if self.agent_loc.y > 1:
                        self.agent_loc.y -= 1
                
                safe_location = (self.agent_loc.x,self.agent_loc.y)
                if safe_location not in self.safe_locations:
                    self.safe_locations.append(safe_location)
                    
        if self.latest_action == 'turn_left':
            if self.orientation == 'RIGHT':
                self.orientation = 'UP'
            elif self.orientation == 'UP':
                self.orientation = 'LEFT'
            elif self.orientation == 'LEFT':
                self.orientation = 'DOWN'
            elif self.orientation == 'DOWN':
                self.orientation = 'RIGHT'

        if self.latest_action == 'turn_right':
            if self.orientation == 'RIGHT':
                self.orientation = 'DOWN'
            elif self.orientation == 'DOWN':
                self.orientation = 'LEFT'
            elif self.orientation == 'LEFT':
                self.orientation = 'UP'
            elif self.orientation == 'UP':
                self.orientation = 'RIGHT'
                
#to update latest percept:    
    def perceive(self, percept):
        self.latest_percept = percept
           

 #reinitialize the agent for the new game:
    def new_game(self,grid):
        return BeelineAgent(grid)


#to plot networkX graph:
    def plot_graph(self, graph):
        import networkx as nx
        import matplotlib.pyplot as plt
        pos = {(x,y):(x,y) for x,y in graph.nodes()}
        nx.draw(graph, pos=pos, node_color='lightgreen', with_labels=True, node_size=600)
        plt.show()
        

#to build the shortest safe path back
    def build_path_back(self) ->list:
        import networkx as nx
        if self.has_gold:
                safe_nodes = self.safe_locations
                unsafe_nodes = [node for node in list(self.graph.nodes) if node not in safe_nodes]
                self.graph.remove_nodes_from(unsafe_nodes) #remove all nodes with unknown safety
                start_node = (self.agent_loc.x,self.agent_loc.y)
                end_node = (1,1)
                self.shortest_path = nx.astar_path(self.graph, start_node, end_node)
                #remove nodes that are not on the shortest path:
                remove_nodes=[node for node in list(self.graph.nodes) if node not in self.shortest_path]
                self.path_back = self.graph
                self.path_back.remove_nodes_from(remove_nodes) 
                self.plot_graph(self.path_back) #visualize the shortest path
                print(self.shortest_path)
                self.shortest_path.pop(0) #to remove current location
                return self.shortest_path
            

#to identify if change of orientation is required: 
    def find_target_orientation(self,target_loc,current_loc) ->str:
        target_location = Location(*target_loc)
        current_loc = Location(*current_loc)
        
        if target_location.x==current_loc.x+1 and target_location.y==current_loc.y:
            self.target_orientation = "RIGHT"
        elif target_location.x==current_loc.x-1 and target_location.y==current_loc.y:
            self.target_orientation = "LEFT"
        elif target_location.x==current_loc.x and target_location.y==current_loc.y+1:
            self.target_orientation = "UP"
        elif target_location.x==current_loc.x and target_location.y==current_loc.y-1:
            self.target_orientation = "DOWN"
        return self.target_orientation


#to execute change of orientation:
    def change_orientation(self,current_orientation, target_orientation) ->str:
        if target_orientation == "DOWN":
            if current_orientation == "UP" or current_orientation == "RIGHT":
                self.latest_action = "turn_right"
            elif current_orientation == "LEFT":
                self.latest_action = "turn_left"

        elif target_orientation == "UP":
            if current_orientation == "DOWN" or current_orientation == "LEFT":
                self.latest_action = "turn_right"
            elif current_orientation == "RIGHT":
                self.latest_action = "turn_left"

        elif target_orientation == "RIGHT":
            if current_orientation == "LEFT" or current_orientation == "UP":
                self.latest_action = "turn_right"
            elif current_orientation == "DOWN":
                self.latest_action = "turn_left"

        elif target_orientation == "LEFT":
            if current_orientation == "RIGHT" or current_orientation == "DOWN":
                self.latest_action = "turn_right"
            elif current_orientation == "UP":
                self.latest_action = "turn_left"
        return self.latest_action
                

In [4]:
def game_engine(agent, env=WumpusEnvironment()):
    new_game = "Y"
    while new_game in["Y","y"]: 
        
        #since agent is random, we need a condition to break the while loop:
        if agent.agent_type in ['naive_agent','beeline_agent']:
            max_moves = 40
            i=1
            
            while not env.game_over  and i<=max_moves+1:
                print(f"Move {i}.")
                env.print_world() 
                current_percept = env.get_percepts_and_score()
                agent.perceive(current_percept)
                next_action=agent.next_action()
                print(f"next action: {next_action}")
                env.take_action(next_action)
                i+=1
       
        else:    
            while not env.game_over:
                env.print_world() 
                current_percept = env.get_percepts_and_score()
                agent.perceive(current_percept)
                next_action=agent.next_action()
                print(f"next action: {next_action}")
                env.take_action(next_action)

        new_game= input("Do you want to play again? Y or N\n")
        if new_game in["Y","y"]:
            env=WumpusEnvironment()
            agent = agent.new_game(env.grid)
        else:
            print("Thank you for playing!")       

In [ ]:
game_engine(BeelineAgent(4)) 

Move 1.
+---+---+---+---+
|   |   |   |   |
|   |   |   |   |
+---+---+---+---+
|   |   |   | G |
|   |   |   |   |
+---+---+---+---+
|   |   |W  |   |
|   |   |   |   |
+---+---+---+---+
|   |   |   |   |
| A>|   |   |   |
+---+---+---+---+
[0]

Selected action: turn_left
next action: turn_left
Move 2.
+---+---+---+---+
|   |   |   |   |
|   |   |   |   |
+---+---+---+---+
|   |   |   | G |
|   |   |   |   |
+---+---+---+---+
|   |   |W  |   |
|   |   |   |   |
+---+---+---+---+
|   |   |   |   |
| A^|   |   |   |
+---+---+---+---+
[-1]

Selected action: forward
next action: forward
Move 3.
+---+---+---+---+
|   |   |   |   |
|   |   |   |   |
+---+---+---+---+
|   |   |   | G |
|   |   |   |   |
+---+---+---+---+
|   |   |W  |   |
| A^|   |   |   |
+---+---+---+---+
|   |   |   |   |
|   |   |   |   |
+---+---+---+---+
[-2]

Selected action: turn_left
next action: turn_left
Move 4.
+---+---+---+---+
|   |   |   |   |
|   |   |   |   |
+---+---+---+---+
|   |   |   | G |
|   |   |   |

class NaiveAgent():
    def __init__(self):
        self.agent_type = 'naive_agent'
        self.all_actions = ['forward', 'turn_left', 'turn_right', 'grab','shoot', 'climb']
        self.legal_actions = self.all_actions
        self.latest_percept = None
        self.score = None
        
    def next_action(self):
        import random
        return random.choice(self.legal_actions)
    
    def perceive(self, percept):
        self.latest_percept = percept
    